In [3]:
import pandas as pd
import os
import re
import json
from multiprocessing import Process

In [4]:
! ls /data/shya6478/fixed/data1.json_156

ls: cannot access /data/shya6478/fixed/data1.json_156: No such file or directory


In [5]:
!mkdir /data/shya6478/fixed-prep/

mkdir: cannot create directory '/data/shya6478/fixed-prep/': File exists


In [6]:
import ast
str_to_obj = lambda s: ast.literal_eval(s)

In [7]:
def prep(fileindex):
    motherdf = pd.DataFrame(columns=['tweet_id', 'tweet_text', 'tweet_tags', 'tweet_created_at',
       'tweet_reply_count', 'tweet_in_reply_to_status_id',
       'tweet_in_reply_to_user_id', 'place_id', 'place_country', 'place_name',
       'user_id', 'user_name', 'user_friends_count', 'user_favourites_count',
       'user_created_at', 'user_verified', 'user_statuses_count',
       'user_followers_count', 'user_default_profile_image'])
    fldr = f'/data/shya6478/07/{fileindex}/'
    for filename in os.listdir(fldr):
        path = fldr+filename
        df = pd.read_csv(path,lineterminator='\n')
        # prepare
        for index, t in df.iterrows():
            processed = processed_tweet(t, index)
            row = create_clean_twt(t, processed)
            motherdf = motherdf.append(row,ignore_index=True)
    motherdf.to_csv(f'/data/shya6478/fixed-prep/data{fileindex}.csv')
    print(f'$$$ Saved#/data{fileindex}.csv')
    return
# df[df['in_reply_to_user_id'].notna()].to_csv('/data/shya6478/only_rpl.csv', single_file = True)

In [8]:
import preprocessor
clean_place = lambda pl: {'id': pl['id'], 'country': pl['country_code'], 'name': pl['name']}
def clean_user(u):
    u = str_to_obj(u)
    wanted_keys = ('id',
                   'screen_name',
                   'friends_count',
                   'favourites_count',
                   'created_at',
                   'verified',
                   'statuses_count',
                   'followers_count')
    del_keys = u.keys()-wanted_keys
    del_values = [u.pop(k) for k in del_keys]
    return u
def clean_tweet(text): 
    t=text.lower()
    tags = preprocessor.parse(t).hashtags
    if tags is not None: 
        tags=[tg.match for tg in preprocessor.parse(t).hashtags]
    return {'text':preprocessor.clean(t), 'tags':tags}
def processed_tweet(t,inx):
    twt = clean_tweet(t['text'])
    usr = clean_user(t['user'])
    try:
        plc = clean_place(str_to_obj(t['place']))
    except:
        plc = {'id': None, 'country': None, 'name':None }
    return {'text':twt['text'], 'tags':twt['tags'], 'user':usr, 'place':plc} #, 'sentiment': sentiment(twt['text'])}

In [9]:
def create_clean_twt(t, processed):
    return {
        'tweet_id':t['id'],
        'tweet_text':processed['text'],
        'tweet_tags':processed['tags'],
        'tweet_created_at':t['created_at'],
        'tweet_in_reply_to_status_id':t['in_reply_to_status_id'],
        'tweet_in_reply_to_user_id':t['in_reply_to_user_id'],
        'place_id':processed['place']['id'],
        'place_country':processed['place']['country'],
        'place_name':processed['place']['name'],
        'user_id':processed['user']['id'],
        'user_name':processed['user']['screen_name'],
        'user_friends_count':processed['user']['friends_count'],
        'user_favourites_count':processed['user']['favourites_count'],
        'user_created_at':processed['user']['created_at'],
        'user_verified':processed['user']['verified'],
        'user_statuses_count':processed['user']['statuses_count'],
        'user_followers_count':processed['user']['followers_count']
    }

In [10]:
import dask
import dask.dataframe as dd

def prepdask(fileindex):
    motherdf = pd.DataFrame(columns=['tweet_id','tweet_text', 'tweet_tags', 'tweet_created_at', 'tweet_in_reply_to_status_id',
       'tweet_in_reply_to_user_id', 'place_id', 'place_country', 'place_name',
       'user_id', 'user_name', 'user_friends_count', 'user_favourites_count',
       'user_created_at', 'user_verified', 'user_statuses_count',
       'user_followers_count'])
    fldr = f'/data/shya6478/fixed/{fileindex}/*'
    df = dd.read_csv(fldr,lineterminator='\n')
    part = 1
    # prepare
    for index, t in df.iterrows():
        if index==0:print(str(fileindex), end=',')
        processed = processed_tweet(t, index)
        row = create_clean_twt(t, processed)
        motherdf = motherdf.append(row,ignore_index=True)
        if len(motherdf.index) > 100000:
            part = part + 1
            motherdf.to_csv(f'/data/shya6478/fixed-prep/data{fileindex}-part{part}.csv')
            print(f'$$$ Saved /data{fileindex}-part{part}.csv')
            motherdf.drop(motherdf.index, inplace=True)
    motherdf.to_csv(f'/data/shya6478/fixed-prep/data{fileindex}.csv')
#     motherdf.to_feather(f'/data/shya6478/prep/data{fileindex}.ftr')
    print(f'$$$ Saved#/data{fileindex}.ftr')
    return

In [14]:
from threading import Thread
for i in [10]:#range(12):
    print(f'starting p#{i}')
#     Process(target=prepdask, args=(i,)).start()
    Thread(target=prepdask, args=(i+1, )).start()

starting p#10
11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11

https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d